In [63]:
from PIL import Image
import json
import pandas as pd

# Create GIFs

In [64]:
# Specify the JSON file path
filename = 'data.json'

# Open the JSON file and load its contents into a dictionary
with open(filename, 'r') as file:
    data = json.load(file)


for round in data:
    list_frames = data[round]
    # Add strings to create filenames 
    fileNames = ['output_' + str(item) + '.jpg' for item in list_frames['frames']] 
    
    print(fileNames)
    

    # Create a list to store the images
    processed_images = []

    # Iterate over the filenames and open each image
    for filename in fileNames:
        try:
            img = Image.open('export/'+ filename)
            # Resize to half of the size
            resized_img = img.resize((img.width // 2, img.height // 2))  # Resize the image
            processed_images .append(resized_img)
        except:
            print('missing frame:' + filename)

    durations = [1/60*1000] * len(processed_images)
    # Save the images as a GIF
    processed_images[0].save(str(round)+'.gif', save_all=True, append_images=processed_images [1:], loop=0, duration=durations)

    print("GIF " + str(round) + " created successfully!")



['output_0.jpg', 'output_1.jpg', 'output_2.jpg', 'output_3.jpg', 'output_4.jpg', 'output_5.jpg', 'output_6.jpg', 'output_7.jpg', 'output_8.jpg', 'output_9.jpg', 'output_10.jpg', 'output_11.jpg', 'output_12.jpg', 'output_13.jpg', 'output_14.jpg', 'output_15.jpg', 'output_16.jpg', 'output_17.jpg', 'output_18.jpg', 'output_19.jpg', 'output_20.jpg', 'output_21.jpg', 'output_22.jpg', 'output_23.jpg', 'output_24.jpg', 'output_25.jpg', 'output_26.jpg', 'output_27.jpg', 'output_28.jpg', 'output_29.jpg', 'output_30.jpg', 'output_31.jpg', 'output_32.jpg', 'output_33.jpg', 'output_34.jpg', 'output_35.jpg', 'output_36.jpg', 'output_37.jpg', 'output_38.jpg', 'output_39.jpg', 'output_40.jpg', 'output_41.jpg', 'output_42.jpg', 'output_43.jpg', 'output_44.jpg', 'output_45.jpg', 'output_46.jpg', 'output_47.jpg', 'output_48.jpg', 'output_49.jpg', 'output_50.jpg', 'output_51.jpg', 'output_52.jpg', 'output_53.jpg', 'output_54.jpg', 'output_55.jpg', 'output_56.jpg', 'output_57.jpg', 'output_58.jpg', 'outpu

# Create dataframe

In [67]:
# Specify the JSON file path
filename = 'data.json'

# Open the JSON file and load its contents into a dictionary
with open(filename, 'r') as file:
    data = json.load(file)


# Define the column names
columns = ['Run_ID', 'Bowl from', 'Result', 'Speed', 'Distance bounce', 'Bounce area']

# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=columns)

round_id = 1

exclude_rounds = [16,20]

for round in data.keys():

    if int(round) not in exclude_rounds:
        round_data = data[round]
        
        # Check if hit or miss
        if len(round_data['miss_point']) == 0:
            round_result = 'Hit'
        else:
            round_result = 'Miss'
        
        # Check if came from left or right

        if round_data['before_bounce'][0][0] < 670:
            round_from = 'Left'
        else:
            round_from = 'Right'

        # Check distance from bounce point to stumps
        round_d_bounce = round_data['bounce_d_meters'][0]
        if round_data['bounce_d_meters'][0] < 2:
            round_area = 'Yorker'
        elif round_data['bounce_d_meters'][0] < 6:
            round_area = 'Full length'
        elif round_data['bounce_d_meters'][0] < 8:
            round_area = 'Good length'
        else:
            round_area = 'Short'
        
        round_speed = round_data['speed'][0] 
        
        new_row = {'Run_ID': 'RUN_' + str(round_id), 'Bowl from': round_from,'Result': round_result, 'Speed': round_speed, 
                   'Distance bounce': round_d_bounce,'Bounce area': round_area }
        df = df.append(new_row, ignore_index=True)

        round_id +=1

# Export DataFrame to Excel
df.to_excel('data.xlsx', index=False)

print(df)


   Run_ID Bowl from Result       Speed  Distance bounce  Bounce area
0       1     Right    Hit  108.390739         7.152865  Good length
1       2     Right    Hit  134.246506         4.735814  Full length
2       3      Left    Hit  146.906077         6.933642  Good length
3       4      Left   Miss  113.374857         7.331630  Good length
4       5     Right    Hit  120.298957         1.368677       Yorker
5       6      Left   Miss  118.678117         5.889028  Full length
6       7     Right    Hit  136.206355         5.221225  Full length
7       8      Left    Hit  133.582713         3.548107  Full length
8       9      Left    Hit  125.322778         6.715607  Good length
9      10     Right    Hit  143.941974         5.350428  Full length
10     11      Left   Miss  124.195159         6.906319  Good length
11     12      Left    Hit  142.521980         0.546548       Yorker
12     13     Right    Hit  130.300159         3.821653  Full length
13     14      Left    Hit  131.83

# Create summary heatmap

In [71]:
import cv2
import numpy as np

# Load the JPG image
image = cv2.imread('bg_heatmap.jpg')

# Add field and batting area overlay to image
color_field = (0,100,0)
color_bat_area = (0,0,0)
color_bounced = (255,0,0)
color_batted = (0,255,0)
color_missed = (0,0,255)

batting_area = [[ 411,  524],
                [411, 150],
                [955, 142],
                [ 955,  514]]

inside_field =  [[ 210,  720],
                [ 411,  524],
                [ 955,  514],
                [1182,  720]]

overlay = image.copy()


overlay = cv2.fillPoly(overlay, [np.array(batting_area).reshape((-1, 1, 2))] , color_bat_area)
overlay = cv2.fillPoly(overlay, [np.array(inside_field).reshape((-1, 1, 2))] , color_field)


alpha = 0.25  # Transparency factor.

# Set overlay over the image
image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)  

# Add all bounce batting and missing points to the image + their projections

# Specify the JSON file path
filename = 'data.json'

# Open the JSON file and load its contents into a dictionary
with open(filename, 'r') as file:
    data = json.load(file)

exclude_rounds = [16,20]

for round in data.keys():

    if int(round) not in exclude_rounds:
        round_data = data[round]

        ###### BOUNCE POINTS

        bounce_point = round_data['bounce_point'][0]
        bounce_point_proj = round_data['bounce_point_proj'][0]

        image = cv2.circle(image, (int(bounce_point[0]),int(bounce_point[1])), 5, color_bounced, -1)
        image = cv2.circle(image, (int(bounce_point_proj[0]),int(bounce_point_proj[1])), 2, color_bounced, -1)
        if 'bat_point' in round_data:
            bat_point = round_data['bat_point'][0]
            bat_point_proj = round_data['bat_point_proj'][0]

            image = cv2.circle(image, (int(bat_point[0]),int(bat_point[1])), 5, color_batted, -1)
            image = cv2.circle(image, (int(bat_point_proj[0]),int(bat_point_proj[1])), 2, color_batted, -1)
        if len(round_data['miss_point']) != 0:
            miss_point = round_data['miss_point'][0]
            miss_point_proj = round_data['miss_point_proj'][0]

            image = cv2.circle(image, (int(miss_point[0]),int(miss_point[1])), 5, color_missed, -1)
            image = cv2.circle(image, (int(miss_point_proj[0]),int(miss_point_proj[1])), 2, color_missed, -1) 


# Display the image
cv2.imshow('Image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

cv2.imwrite('summary_heatmap.jpg', image)


True

In [56]:
round_data

{'frames': [822,
  823,
  824,
  825,
  826,
  827,
  828,
  829,
  830,
  831,
  832,
  833,
  834,
  835,
  836,
  837,
  838,
  839,
  840,
  841,
  842,
  843,
  844,
  845,
  846,
  847,
  848,
  849,
  850,
  851,
  852,
  853,
  854,
  855,
  856,
  857,
  858,
  859,
  860,
  861,
  862,
  863,
  864,
  865,
  866,
  867,
  868,
  869,
  870,
  871,
  872,
  873,
  874,
  875,
  876,
  877,
  878,
  879,
  880],
 'complete_trace': [[362.0, 25.5],
  [379.0, 55.0],
  [411.0, 120.5],
  [nan, nan],
  [450.5, 206.5],
  [472.5, 256.0],
  [491.5, 301.5],
  [501.5, 323.0],
  [519.5, 365.5],
  [535.0, 401.5],
  [543.0, 418.5],
  [556.0, 456.0],
  [570.5, 490.0],
  [575.0, 505.0],
  [585.0, 538.5],
  [593.5, 566.0],
  [597.5, 575.0],
  [606.0, 550.0],
  [611.0, 515.0],
  [615.0, 502.0],
  [619.0, 476.5],
  [624.5, 453.5],
  [627.0, 443.0],
  [631.0, 421.0],
  [632.0, 401.5],
  [633.5, 393.5],
  [636.0, 379.5],
  [637.0, 367.0],
  [638.5, 359.5],
  [638.5, 348.0],
  [nan, nan],
  [nan, na